# Analysis of causal discovery algorithm on synthetic data

In [2]:
import sys
sys.path.append('..')
import os
import numpy as np
import pickle
from tqdm.notebook import tqdm
from synthetic_data import generate_data
from causal import causal_eval
from plots import plot_SHD, plot_samples

from warnings import filterwarnings
filterwarnings('ignore')

### Parameters

In [7]:
# number of trials and permutations
n_trials = 100
n_perms = 1000

# number of samples and number of points functional data samples are (randomly) observed and discretised
n_samples = [200]
n_obs = 100
n_preds = 100

# define discretised mesh of points
upper_limit = 1
pred_points = np.linspace(0, upper_limit, n_preds)

# data paramterers
periods = [0.1]
n_basis = 3
sd = 1

# statistical significance level
alpha = 0.05

In [8]:
# create folders to save results
if not os.path.exists('results'):
    os.mkdir('results')

if not os.path.exists('results/causal'):
    os.mkdir('results/causal')

Parameters specific for evaluation on synthetic data

In [9]:
test = 'joint'
n_vars = 10
prob = 0.5
cd_type = 'combined'

# historical dependence is easier to detect the higher a is
a_list = [10, 1, 0.1]

# regression parameters
n_intervals = 12
analyse = False

# constraint parameters
lambs = [1e-5, 1e-4, 1e-3]
#lambs = 1e-3
n_steps = 50
n_pretests = 100

linear = 0
bll1, bul1, bll2, bul2, bll3, bul3, bll4, bul4 = -1, -1, -1, -1, -1, -1, -1, -1
#bll1, bul1, bll2, bul2, bll3, bul3, bll4, bul4 = 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# saving evaluation metrics
precisions_dict = {}
recalls_dict = {}
f1_scores_dict = {}
SHDs_dict = {}
GLs_dict = {}
averages_dict = {}

# saving DAGs + p-values
edges_dict = {}
DAGs_dict = {}
p_values_dict = {}

for p in periods:
    print('Period T:', p)
    precisions_dict[p] = {}
    recalls_dict[p] = {}
    f1_scores_dict[p] = {}
    SHDs_dict[p] = {}
    GLs_dict[p] = {}
    averages_dict[p] = {}
    edges_dict[p] = {}
    DAGs_dict[p] = {}
    p_values_dict[p] = {}
    
    for n_sample in tqdm(n_samples):
        print('Sample size:', n_sample)

        precisions_dict[p][n_sample] = {}
        recalls_dict[p][n_sample] = {}
        f1_scores_dict[p][n_sample] = {}
        SHDs_dict[p][n_sample] = {}
        GLs_dict[p][n_sample] = {}
        averages_dict[p][n_sample] = {}
        edges_dict[p][n_sample] = {}
        DAGs_dict[p][n_sample] = {}
        p_values_dict[p][n_sample] = {}

        for i, a in enumerate(a_list):
            print('a:', a)

            precisions_dict[p][n_sample][a] = []
            recalls_dict[p][n_sample][a] = []
            f1_scores_dict[p][n_sample][a] = []
            SHDs_dict[p][n_sample][a] = []
            GLs_dict[p][n_sample][a] = []
            averages_dict[p][n_sample][a] = []
            edges_dict[p][n_sample][a] = []
            DAGs_dict[p][n_sample][a] = []
            p_values_dict[p][n_sample][a] = []

            # generate synthetic data
            edges, X = generate_data(dep=test, n_samples=n_sample, n_trials=n_trials, n_obs=n_obs, n_preds=n_preds, period=p, n_vars=n_vars, a=a, upper_limit=upper_limit, n_basis=n_basis, sd=sd, prob=prob, bll1=bll1, bul1=bul1, bll2=bll2, bul2=bul2, bll3=bll3, bul3=bul3, bll4=bll4, bul4=bul4, linear=linear)
            
            # conduct n trials
            precisions, recalls, f1_scores, SHDs, GLs, CPDAGs, p_values = causal_eval(cd_type=cd_type, X_dict=X, edges_dict=edges, upper_limit=upper_limit, n_preds=n_preds, n_intervals=n_intervals, n_trials=n_trials, n_perms=n_perms, alpha=alpha, K='K_ID', lambs=lambs, analyse=analyse)

            precisions_dict[p][n_sample][a].append(precisions)
            recalls_dict[p][n_sample][a].append(recalls)
            f1_scores_dict[p][n_sample][a].append(f1_scores)
            SHDs_dict[p][n_sample][a].append(SHDs)
            GLs_dict[p][n_sample][a].append(GLs)

            edges_dict[p][n_sample][a].append(edges)
            DAGs_dict[p][n_sample][a].append(CPDAGs)
            p_values_dict[p][n_sample][a].append(p_values)

            # calculate average precision, recall and F1-score
            avg_precision = np.mean(precisions_dict[p][n_sample][a])
            avg_recall = np.mean(recalls_dict[p][n_sample][a])
            avg_f1_score = np.mean(f1_scores_dict[p][n_sample][a])
            avg_SHDs = np.mean(SHDs_dict[p][n_sample][a])
            avg_GLs = np.mean(GLs_dict[p][n_sample][a])

            averages_dict[p][n_sample][a].append([avg_precision, avg_recall, avg_f1_score, avg_SHDs, avg_GLs])

            print('Average SHD:', avg_SHDs)
            print('Average GL:', avg_GLs)
            print('----------')
        print('----------')
    print('----------')

precision_causal = open('results/causal/precision_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(precisions_dict, precision_causal)
precision_causal.close()
recall_causal = open('results/causal/recall_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(recalls_dict, recall_causal)
recall_causal.close()
f1_causal = open('results/causal/f1_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(f1_scores_dict, f1_causal)
f1_causal.close()
SHD_causal = open('results/causal/shd_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(SHDs_dict, SHD_causal)
SHD_causal.close()
GL_causal = open('results/causal/GL_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(GLs_dict, GL_causal)
GL_causal.close()
averages_causal = open('results/causal/averages_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(averages_dict, averages_causal)
averages_causal.close()

edges_causal = open('results/causal/edges_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(edges_dict, edges_causal)
edges_causal.close()
DAGs_causal = open('results/causal/DAGs_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(DAGs_dict, DAGs_causal)
DAGs_causal.close()
pvalues_causal = open('results/causal/p_values_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'wb')
pickle.dump(p_values_dict, pvalues_causal)
pvalues_causal.close()

Period T: 0.1


  0%|          | 0/1 [00:00<?, ?it/s]

Sample size: 200
a: 10


  0%|          | 0/100 [00:00<?, ?it/s]

Finding optimal lambda:
Evaluating lambda = 1e-05 (value 1 of 3)...


...

...Completed with a rejection rate of 0.04.
Evaluating lambda = 0.0001 (value 2 of 3)...


  0%|          | 0/100 [00:00<?, ?it/s]

...Completed with a rejection rate of 0.02.
Evaluating lambda = 0.001 (value 3 of 3)...


  0%|          | 0/100 [00:00<?, ?it/s]

...Completed with a rejection rate of 0.04.
Optimal lambda is 0.001 and resulted in a rejection rate of 0.04.


In [ ]:
cd_type = 'constraint'
n_vars = 3
causal_results = pickle.load(open('results/causal/shd_{}_{}_{}.pkl'.format(cd_type, n_vars, n_samples[0]), 'rb'))

In [ ]:
causal_results